In [1]:
import nglview as nv
import pytraj as pt
import random

from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

import xml.etree.ElementTree as ElementTree

_ColormakerRegistry()

In [2]:
def load_sequence(path):
    with open(path) as f:
        fasta = f.read().strip()
    return fasta.split('\n')[-1]
    
seq = load_sequence('proteins/6ct4.fasta')
seq

'PMKKLKLALRLAAKIAPVW'

In [3]:
tree = ElementTree.parse('amber99sb.xml').getroot()
atoms = {}
for c in tree[1]:
    atoms[c.attrib['name']] = [a.attrib['name'] for a in c.findall('Atom')]

In [4]:
mappings = 'ala:A|arg:R|asn:N|asp:D|cys:C|gln:Q|glu:E|gly:G|his:H|ile:I|leu:L|lys:K|met:M|phe:F|pro:P|ser:S|thr:T|trp:W|tyr:Y|val:V'.upper().split('|')
l2aa = dict([m.split(':')[::-1] for m in mappings])

In [5]:
def write_randpos_pdb(path, aa_sequence):
    data = ''
    serial = 1
    for residue_num, residue_letter in enumerate(aa_sequence):
        residue = l2aa[residue_letter]
        atom_names = atoms[residue].copy()
        if residue_num == 0:
            atom_names += ['H2', 'H3']
        if residue_num == len(aa_sequence) - 1:
            atom_names += ['OXT']
        for name in atom_names:
            rand = lambda s=0.1, off=0: (random.random() - 0.5) * 2 * s + off
            if name == 'CA':
                x, y, z = rand(s=0.01, off=residue_num), rand(s=0.01), rand(s=0.01)
            else:
                x, y, z = rand(off=residue_num), rand(), rand()
            
            if len(name) == 3:
                name = ' ' + name
            data += f'{"ATOM":6}{serial:5} {name:^4} {residue:3} {"A":1}{residue_num+1:4}    {x:8.3f}{y:8.3f}{z:8.3f}{1:6.2f}{0:6.2f}           {name.strip()[0]:2}{"":2}\n'
            serial += 1
    with open(path, 'w') as file:
        file.write(data)

write_randpos_pdb('proteins/test.pdb', seq)

In [6]:
pdb = PDBFile('proteins/test.pdb')
#forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
forcefield = ForceField('amber99sb.xml')
print(pdb.topology)

modeller = Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens()

system = forcefield.createSystem(modeller.topology, constraints=HBonds)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

simulation.minimizeEnergy()

steps = 100000
simulation.reporters.append(PDBReporter('proteins/output.pdb', steps // 100))
simulation.reporters.append(StateDataReporter(stdout, steps // 10, step=True, potentialEnergy=True, temperature=True))

<Topology; 1 chains, 19 residues, 335 atoms, 338 bonds>


In [38]:
simulation.step(steps)

1501000,1502.035955765988,285.129008361142
1502000,1529.6546436949693,303.02358330221585
1503000,1504.1421321497528,285.4968200822799
1504000,1476.161498031091,299.73407827992986
1505000,1536.2268045794885,279.5454585883828
1506000,1566.5233974807902,275.0395557783844
1507000,1487.0825074289473,310.2756908912489
1508000,1543.1708650164844,294.3094630264019
1509000,1581.4481445132665,289.1442138785816
1510000,1558.9574914347281,296.0269221042342
1511000,1476.3168614721137,307.8128742321551
1512000,1464.6329913201357,280.12553278161187
1513000,1552.9813726592156,284.11316378563987
1514000,1608.3421530175956,301.1939979115615
1515000,1592.0844765241784,286.90432379974436
1516000,1555.2661501637135,278.4455791922555
1517000,1545.1199783232655,286.50244381129477
1518000,1483.2081041650522,310.35068607033327
1519000,1537.298137583895,322.1890227439045
1520000,1504.5484429506205,322.0264532986738
1521000,1541.7409272009704,298.2725242347366
1522000,1523.3919969392084,291.8694150550031
1523000

In [39]:
view = nv.show_pytraj(pt.load('proteins/output.pdb'))
view

NGLWidget(max_frame=15999)

In [9]:
view.close()